## PFG Bank: Credit Card Design

* Team-lead GitLab id:
* Group name:
* Team member names:

## Setup

Please complete this Jupyter notebook by answering the questions in `pfg-bank.pdf` on Canvas (week10/). Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML from the Jupyter Notebook file without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the final group assignment for MGTA 455 and you will be using git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". git will not decide for you who's change to accept so the team-lead will have to determine which edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and push them to GitLab. Make "pull first" a habit!

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import os
import numpy as np
import pyrsm as rsm
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

## Question answers

In [2]:
exhibit_1 = pd.read_excel('data/exhibits.xls', sheet_name = 'exhibit1')
exhibit_2 = pd.read_excel('data/exhibits.xls', sheet_name = 'exhibit2')

In [3]:
exhibit_1

,date,apr,fixed_var,annual_fee,visamc,nr_mailed,non_resp,resp,bk_score,average_bk
0,April,16.8,Fixed,20,MC,167000,165467,1533,200,210
1,April,16.8,Fixed,0,MC,81000,78104,2896,200,210
2,April,19.8,Fixed,20,MC,143000,142410,590,200,210
3,April,19.8,Fixed,0,MC,100000,97948,2052,200,210
4,September,14.9,Fixed,20,Visa,177000,172671,4329,250,255
5,September,14.9,Variable,20,Visa,170000,166996,3004,250,255
6,September,16.8,Fixed,20,Visa,255000,252017,2983,250,255
7,September,19.8,Fixed,20,Visa,35000,34825,175,250,255
8,September,16.8,Fixed,0,Visa,65000,62484,2516,250,255
9,September,19.8,Fixed,0,Visa,95000,92885,2115,250,255


In [4]:
exhibit_1["resp_rate"] = exhibit_1['resp'] / exhibit_1['nr_mailed']
exhibit_1

,date,apr,fixed_var,annual_fee,visamc,nr_mailed,non_resp,resp,bk_score,average_bk,resp_rate
0,April,16.8,Fixed,20,MC,167000,165467,1533,200,210,0.009180
1,April,16.8,Fixed,0,MC,81000,78104,2896,200,210,0.035753
2,April,19.8,Fixed,20,MC,143000,142410,590,200,210,0.004126
3,April,19.8,Fixed,0,MC,100000,97948,2052,200,210,0.020520
4,September,14.9,Fixed,20,Visa,177000,172671,4329,250,255,0.024458
5,September,14.9,Variable,20,Visa,170000,166996,3004,250,255,0.017671
6,September,16.8,Fixed,20,Visa,255000,252017,2983,250,255,0.011698
7,September,19.8,Fixed,20,Visa,35000,34825,175,250,255,0.005000
8,September,16.8,Fixed,0,Visa,65000,62484,2516,250,255,0.038708
9,September,19.8,Fixed,0,Visa,95000,92885,2115,250,255,0.022263


In [5]:
exhibit1_melt = pd.melt(
    exhibit_1,
    id_vars=["apr", "fixed_var", "annual_fee", "bk_score", "nr_mailed"],
    value_vars=["resp", "non_resp"],
    var_name="resp",
    value_name="freq",
)
exhibit1_melt["resp_sale"] = rsm.ifelse(exhibit1_melt.resp == "resp", 1, 0)

#exhibit1_melt["date"] = exhibit1_melt["date"].astype('category')
exhibit1_melt["apr"] = exhibit1_melt["apr"].astype('category')
exhibit1_melt["fixed_var"] = exhibit1_melt["fixed_var"].astype('category')
exhibit1_melt["annual_fee"] = exhibit1_melt["annual_fee"].astype('category')
exhibit1_melt["bk_score"] = exhibit1_melt["bk_score"].astype('category')
#exhibit1_melt["visamc"] = exhibit1_melt["visamc"].astype('category')

exhibit1_melt

,apr,fixed_var,annual_fee,bk_score,nr_mailed,resp,freq,resp_sale
0,16.8,Fixed,20,200,167000,resp,1533,1
1,16.8,Fixed,0,200,81000,resp,2896,1
2,19.8,Fixed,20,200,143000,resp,590,1
3,19.8,Fixed,0,200,100000,resp,2052,1
4,14.9,Fixed,20,250,177000,resp,4329,1
5,14.9,Variable,20,250,170000,resp,3004,1
6,16.8,Fixed,20,250,255000,resp,2983,1
7,19.8,Fixed,20,250,35000,resp,175,1
8,16.8,Fixed,0,250,65000,resp,2516,1
9,19.8,Fixed,0,250,95000,resp,2115,1


In [6]:
exhibit1 = rsm.table2data(exhibit1_melt, "freq").reset_index(drop = True)

exhibit1

,apr,fixed_var,annual_fee,bk_score,nr_mailed,resp,freq,resp_sale
0,16.8,Fixed,20,200,167000,resp,1533,1
1,16.8,Fixed,20,200,167000,resp,1533,1
2,16.8,Fixed,20,200,167000,resp,1533,1
3,16.8,Fixed,20,200,167000,resp,1533,1
4,16.8,Fixed,20,200,167000,resp,1533,1
...,...,...,...,...,...,...,...,...
1519995,16.8,Fixed,20,150,50000,non_resp,49627,0
1519996,16.8,Fixed,20,150,50000,non_resp,49627,0
1519997,16.8,Fixed,20,150,50000,non_resp,49627,0
1519998,16.8,Fixed,20,150,50000,non_resp,49627,0


In [7]:
lr = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score",
    family=Binomial(link=logit()),
    data=exhibit1,
).fit()
rsm.or_ci(lr)


,index,OR,OR%,2.5%,97.5%,p.values,
1,apr[T.16.8],0.471,-52.9%,0.454,0.489,< .001,***
2,apr[T.19.8],0.257,-74.3%,0.245,0.269,< .001,***
3,fixed_var[T.Variable],0.741,-25.9%,0.711,0.772,< .001,***
4,annual_fee[T.20],0.290,-71.0%,0.281,0.298,< .001,***
5,bk_score[T.200],1.232,23.2%,1.174,1.293,< .001,***
6,bk_score[T.250],1.425,42.5%,1.372,1.480,< .001,***


In [8]:
lr = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score",
    family=Binomial(link=logit()),
    data=exhibit1_melt,
    freq_weights=exhibit1_melt.freq
).fit()
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,apr[T.16.8],0.471,-52.9%,0.454,0.489,< .001,***
2,apr[T.19.8],0.257,-74.3%,0.245,0.269,< .001,***
3,fixed_var[T.Variable],0.741,-25.9%,0.711,0.772,< .001,***
4,annual_fee[T.20],0.290,-71.0%,0.281,0.298,< .001,***
5,bk_score[T.200],1.232,23.2%,1.174,1.293,< .001,***
6,bk_score[T.250],1.425,42.5%,1.372,1.480,< .001,***


In [9]:
exhibit1_melt["pred_partial"] = lr.predict(exhibit1_melt)
exhibit1_melt['percent'] = exhibit1_melt['freq'] / exhibit1_melt['nr_mailed']

In [10]:
dct = rsm.levels_list(exhibit1_melt[["apr", "fixed_var", "annual_fee", "bk_score"]])
dct

{'apr': [16.8, 19.8, 14.9],
 'fixed_var': ['Fixed', 'Variable'],
 'annual_fee': [20, 0],
 'bk_score': [200, 250, 150]}

In [11]:
exhibit1_expand = rsm.expand_grid(dct)
exhibit1_expand['apr'] = exhibit1_expand['apr'].astype('category')
exhibit1_expand['fixed_var'] = exhibit1_expand['fixed_var'].astype('category')
exhibit1_expand['annual_fee'] = exhibit1_expand['annual_fee'].astype('category')
exhibit1_expand['bk_score'] = exhibit1_expand['bk_score'].astype('category')
exhibit1_expand

,apr,fixed_var,annual_fee,bk_score
0,16.8,Fixed,20,200
1,16.8,Fixed,20,250
2,16.8,Fixed,20,150
3,16.8,Fixed,0,200
4,16.8,Fixed,0,250
5,16.8,Fixed,0,150
6,16.8,Variable,20,200
7,16.8,Variable,20,250
8,16.8,Variable,20,150
9,16.8,Variable,0,200


In [12]:
exhibit1_expand["pred_all"] = lr.predict(exhibit1_expand)
exhibit1_expand

,apr,fixed_var,annual_fee,bk_score,pred_all
0,16.8,Fixed,20,200,0.010029
1,16.8,Fixed,20,250,0.011581
2,16.8,Fixed,20,150,0.008156
3,16.8,Fixed,0,200,0.033799
4,16.8,Fixed,0,250,0.038884
5,16.8,Fixed,0,150,0.027609
6,16.8,Variable,20,200,0.007448
7,16.8,Variable,20,250,0.008604
8,16.8,Variable,20,150,0.006054
9,16.8,Variable,0,200,0.025257


In [13]:
exhibit_2

,offer,apr,fixed_var,annual_fee,clv150,clv200,clv250
0,1,14.9,Fixed,20,83,63,33
1,2,14.9,Variable,20,93,73,43
2,3,14.9,Fixed,0,52,32,2
3,4,14.9,Variable,0,62,42,12
4,5,16.8,Fixed,20,103,83,53
5,6,16.8,Variable,20,113,93,63
6,7,16.8,Fixed,0,72,52,22
7,8,16.8,Variable,0,82,62,32
8,9,19.8,Fixed,20,131,111,81
9,10,19.8,Variable,20,141,121,91


In [14]:
exhibit2 = pd.melt(exhibit_2, id_vars=["apr", "fixed_var", "annual_fee"], value_vars = ["clv150", "clv200", "clv250"], var_name = ['bk_score'], value_name = 'clv')
exhibit2['bk_score'] = exhibit2['bk_score'].apply(lambda x: 150 if x == 'clv150' else (200 if x == 'clv200' else 250))

In [15]:
exhibit2

,apr,fixed_var,annual_fee,bk_score,clv
0,14.9,Fixed,20,150,83
1,14.9,Variable,20,150,93
2,14.9,Fixed,0,150,52
3,14.9,Variable,0,150,62
4,16.8,Fixed,20,150,103
5,16.8,Variable,20,150,113
6,16.8,Fixed,0,150,72
7,16.8,Variable,0,150,82
8,19.8,Fixed,20,150,131
9,19.8,Variable,20,150,141


In [16]:
clv = exhibit1_expand.merge(exhibit2, how = 'left', on = ['apr', 'fixed_var', 'annual_fee', 'bk_score'])
clv['exp_clv'] = clv['pred_all'] * clv['clv']
print(clv.sort_values(by = ['bk_score', 'exp_clv'], ascending = False))
print(clv['pred_all'].mean())

     apr fixed_var  annual_fee  bk_score  pred_all  clv   exp_clv
16  19.8     Fixed           0       250  0.021593   50  1.079634
22  19.8  Variable           0       250  0.016084   60  0.965060
10  16.8  Variable           0       250  0.029096   32  0.931059
4   16.8     Fixed           0       250  0.038884   22  0.855442
25  14.9     Fixed          20       250  0.024258   33  0.800528
31  14.9  Variable          20       250  0.018083   43  0.777553
34  14.9  Variable           0       250  0.059786   12  0.717426
1   16.8     Fixed          20       250  0.011581   53  0.613799
7   16.8  Variable          20       250  0.008604   63  0.542070
13  19.8     Fixed          20       250  0.006351   81  0.514425
19  19.8  Variable          20       250  0.004712   91  0.428798
28  14.9     Fixed           0       250  0.079057    2  0.158115
27  14.9     Fixed           0       200  0.069096   32  2.211081
33  14.9  Variable           0       200  0.052115   42  2.188838
3   16.8  

In [17]:
round_1 = pd.read_csv('data/round1_result.csv').iloc[0:12, 0:9]
round_1.rename({'APR':"apr", "Fixed/Var.":"fixed_var", "Annual Fee":"annual_fee"}, inplace = True, axis = 1)
round_1['apr'] = round_1['apr'].astype('category')
round_1['fixed_var'] = round_1['fixed_var'].astype('category')
round_1['annual_fee'] = round_1['annual_fee'].astype('category')
round_1['fixed_var'] = round_1["fixed_var"].apply(lambda x: 'Variable' if x == 'Var.' else 'Fixed')
round_1['annual_fee'] = round_1["annual_fee"].apply(lambda x: x[1:])
round_1["150_Sent"] = round_1['150_Sent'].str.replace(',', '').astype(int)
round_1["200_Sent"] = round_1['200_Sent'].str.replace(',', '').astype(int)
round_1["250_Sent"] = round_1['250_Sent'].str.replace(',', '').astype(int)

round1_nr_mailed = pd.melt(
    round_1,
    id_vars=["apr", "fixed_var", "annual_fee","150_Responses", "200_Responses", "250_Responses"],
    value_vars=["150_Sent", "200_Sent", "250_Sent"],
    var_name="bk_score",
    value_name="nr_mailed",
)

round1_nr_mailed = round1_nr_mailed[["apr", "fixed_var", "annual_fee", "bk_score", "nr_mailed"]]


round1_resp = pd.melt(
    round_1,
    id_vars=["apr", "fixed_var", "annual_fee","150_Sent", "200_Sent", "250_Sent",],
    value_vars=["150_Responses", "200_Responses", "250_Responses"],
    var_name="bk_score",
    value_name="resp",
)

round1_resp = round1_resp[['apr', 'fixed_var', 'annual_fee', 'bk_score', 'resp']]

round1_nr_mailed['bk_score'] = round1_nr_mailed['bk_score'].apply(lambda x: x[:3])
round1_resp['bk_score'] = round1_resp['bk_score'].apply(lambda x: x[:3])

round1 = round1_nr_mailed.merge(round1_resp, how = "inner", on = ['apr', 'fixed_var', 'annual_fee', 'bk_score'])
round1["non_resp"] = round1['nr_mailed'] - round1['resp']
round1['annual_fee'] = round1["annual_fee"].apply(lambda x: x[:-1])

round1 = round1[round1['nr_mailed'] != 0].reset_index(drop = True)
round1

,apr,fixed_var,annual_fee,bk_score,nr_mailed,resp,non_resp
0,14.9,Fixed,20,150,9000,137.0,8863.0
1,14.9,Variable,0,150,1225,40.0,1185.0
2,16.8,Variable,20,150,723,2.0,721.0
3,16.8,Fixed,0,150,9000,121.0,8879.0
4,19.8,Fixed,20,150,548,0.0,548.0
5,19.8,Variable,0,150,1949,19.0,1930.0
6,14.9,Fixed,20,200,9000,199.0,8801.0
7,14.9,Variable,0,200,633,24.0,609.0
8,16.8,Variable,20,200,925,3.0,922.0
9,16.8,Fixed,0,200,3535,77.0,3458.0


In [18]:
round1_melt = pd.melt(
    round1,
    id_vars=["apr", "fixed_var", "annual_fee", "bk_score", "nr_mailed"],
    value_vars=["resp", "non_resp"],
    var_name="resp",
    value_name="freq",
)

round1_melt["resp_sale"] = rsm.ifelse(round1_melt.resp == "resp", 1, 0)
round1_melt['bk_score'] = round1_melt['bk_score'].astype('category')

round1_melt

,apr,fixed_var,annual_fee,bk_score,nr_mailed,resp,freq,resp_sale
0,14.9,Fixed,20,150,9000,resp,137.0,1
1,14.9,Variable,0,150,1225,resp,40.0,1
2,16.8,Variable,20,150,723,resp,2.0,1
3,16.8,Fixed,0,150,9000,resp,121.0,1
4,19.8,Fixed,20,150,548,resp,0.0,1
5,19.8,Variable,0,150,1949,resp,19.0,1
6,14.9,Fixed,20,200,9000,resp,199.0,1
7,14.9,Variable,0,200,633,resp,24.0,1
8,16.8,Variable,20,200,925,resp,3.0,1
9,16.8,Fixed,0,200,3535,resp,77.0,1


In [19]:
lr = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score",
    family=Binomial(link=logit()),
    data=round1_melt,
    freq_weights=round1_melt.freq
).fit()
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,apr[T.16.8],0.372,-62.8%,0.302,0.457,< .001,***
2,apr[T.19.8],0.253,-74.7%,0.197,0.324,< .001,***
3,fixed_var[T.Variable],0.883,-11.7%,0.768,1.016,0.082,.
4,annual_fee[T.20],0.350,-65.0%,0.285,0.429,< .001,***
5,bk_score[T.200],1.525,52.5%,1.305,1.782,< .001,***
6,bk_score[T.250],1.671,67.1%,1.415,1.973,< .001,***


In [20]:
dct = rsm.levels_list(round1_melt[["apr", "fixed_var", "annual_fee", "bk_score"]])
dct

{'apr': [14.9, 16.8, 19.8],
 'fixed_var': ['Fixed', 'Variable'],
 'annual_fee': ['20', '0'],
 'bk_score': ['150', '200', '250']}

In [21]:
round1_expand = rsm.expand_grid(dct)
round1_expand['apr'] = round1_expand['apr'].astype('category')
round1_expand['fixed_var'] = round1_expand['fixed_var'].astype('category')
round1_expand['annual_fee'] = round1_expand['annual_fee'].astype('category')
round1_expand['bk_score'] = round1_expand['bk_score'].astype('category')
round1_expand

,apr,fixed_var,annual_fee,bk_score
0,14.9,Fixed,20,150
1,14.9,Fixed,20,200
2,14.9,Fixed,20,250
3,14.9,Fixed,0,150
4,14.9,Fixed,0,200
5,14.9,Fixed,0,250
6,14.9,Variable,20,150
7,14.9,Variable,20,200
8,14.9,Variable,20,250
9,14.9,Variable,0,150


In [22]:
round1_expand["pred_all"] = lr.predict(round1_expand)
round1_expand["annual_fee"] = round1_expand["annual_fee"].astype(int)
round1_expand["bk_score"] = round1_expand["bk_score"].astype(int)
round1_expand

,apr,fixed_var,annual_fee,bk_score,pred_all
0,14.9,Fixed,20,150,0.013743
1,14.9,Fixed,20,200,0.020809
2,14.9,Fixed,20,250,0.022753
3,14.9,Fixed,0,150,0.038313
4,14.9,Fixed,0,200,0.057277
5,14.9,Fixed,0,250,0.062411
6,14.9,Variable,20,150,0.012158
7,14.9,Variable,20,200,0.018425
8,14.9,Variable,20,250,0.020151
9,14.9,Variable,0,150,0.033992


In [23]:
exhibit_2 = pd.read_excel('data/exhibits.xls', sheet_name = 'exhibit2')
exhibit2 = pd.melt(exhibit_2, id_vars=["apr", "fixed_var", "annual_fee"], value_vars = ["clv150", "clv200", "clv250"], var_name = ['bk_score'], value_name = 'clv')
exhibit2['bk_score'] = exhibit2['bk_score'].apply(lambda x: 150 if x == 'clv150' else (200 if x == 'clv200' else 250))
exhibit2['annual_fee'] = exhibit2['annual_fee'].astype(int)
exhibit2['bk_score'] = exhibit2['bk_score'].astype(int)
exhibit2

,apr,fixed_var,annual_fee,bk_score,clv
0,14.9,Fixed,20,150,83
1,14.9,Variable,20,150,93
2,14.9,Fixed,0,150,52
3,14.9,Variable,0,150,62
4,16.8,Fixed,20,150,103
5,16.8,Variable,20,150,113
6,16.8,Fixed,0,150,72
7,16.8,Variable,0,150,82
8,19.8,Fixed,20,150,131
9,19.8,Variable,20,150,141


In [24]:
round1_clv = round1_expand.merge(exhibit2, how = 'left', on = ["apr", "fixed_var", "annual_fee", "bk_score"])
round1_clv['exp_clv'] = round1_clv['pred_all'] * round1_clv['clv']
print(round1_clv.sort_values(by = ['bk_score', 'exp_clv'], ascending = False))
print(round1_clv.sort_values(by = ['bk_score', 'pred_all'], ascending = False))

     apr fixed_var  annual_fee  bk_score  pred_all  clv   exp_clv
35  19.8  Variable           0       250  0.014642   60  0.878505
8   14.9  Variable          20       250  0.020151   43  0.866473
29  19.8     Fixed           0       250  0.016545   50  0.827248
2   14.9     Fixed          20       250  0.022753   33  0.750858
23  16.8  Variable           0       250  0.021378   32  0.684088
11  14.9  Variable           0       250  0.055530   12  0.666359
17  16.8     Fixed           0       250  0.024135   22  0.530972
20  16.8  Variable          20       250  0.007583   63  0.477717
26  19.8     Fixed          20       250  0.005850   81  0.473845
32  19.8  Variable          20       250  0.005171   91  0.470519
14  16.8     Fixed          20       250  0.008576   53  0.454552
5   14.9     Fixed           0       250  0.062411    2  0.124823
10  14.9  Variable           0       200  0.050931   42  2.139111
4   14.9     Fixed           0       200  0.057277   32  1.832874
7   14.9  

In [25]:
round1_clv = round1_clv.sort_values(by = ['bk_score', 'exp_clv'], ascending = False)

clv_250 = round1_clv[round1_clv['bk_score'] == 250].iloc[0:2]
clv_250['ratio'] = clv_250['exp_clv'] / clv_250['exp_clv'].sum()
clv_250

,apr,fixed_var,annual_fee,bk_score,pred_all,clv,exp_clv,ratio
35,19.8,Variable,0,250,0.014642,60,0.878505,0.503447
8,14.9,Variable,20,250,0.020151,43,0.866473,0.496553


In [26]:
clv_200 = round1_clv[round1_clv['bk_score'] == 200].iloc[0:2]
clv_200['ratio'] = clv_200['exp_clv'] / clv_200['exp_clv'].sum()
clv_200

,apr,fixed_var,annual_fee,bk_score,pred_all,clv,exp_clv,ratio
10,14.9,Variable,0,200,0.050931,42,2.139111,0.53855
4,14.9,Fixed,0,200,0.057277,32,1.832874,0.46145


In [27]:
clv_150 = round1_clv[round1_clv['bk_score'] == 150].iloc[0:2]
clv_150['ratio'] = clv_150['exp_clv'] / clv_150['exp_clv'].sum()
clv_150

,apr,fixed_var,annual_fee,bk_score,pred_all,clv,exp_clv,ratio
9,14.9,Variable,0,150,0.033992,62,2.107525,0.514055
3,14.9,Fixed,0,150,0.038313,52,1.992278,0.485945


# Questions

1. Why does Customer Lifetime Value vary with BK score? Why does Customer Lifetime Value vary by product? (See Exhibit 2 to help answer these questions) (3 points)

We found that customers with low BK score tend to have high CLV. That is because people with low BK score are less likely to bankrupt and thus less likely to default. They are potentially more wealthier and have less financial pressures. In the long term, they can continuing provide value to PFG Bank, which lead to higher CLV.
Different products have different financial attributes. Each attribute functionality (like Fixed and varied card type) would use distinct calculation process, and thus leads to different customer CLV. For example, one of the potential scenario we see from exhibit 2 that, for customers with 150 BK score, the CLV of 19.8 APR + Fixed + 20 Annual Fee is 131, but the CLV of 19.8 APR + Variable + 20 Annual Fee is 141. Since we know that a variable-rate card could pass along most of the interest-rate risk to the customer, thus making PFB Bank more profitble if customers choose variable-rate card and leading to higher CLV.

2. Are predictive models estimated on historical data useful in this case? If so, why? If not, why not? (4 points)

Yes and no. Yes, the historical data is useful to train the model and help us to understand and predict the customer behaviors, comparing to estimate a result without any historical data. But the simulation results are unsatisfying (low profit and low customer response rate) when we used logistic regression model trained by historical data. Moreover, the predicted result trained by historical data does not align with that by round 1 data. In conclusion, in terms of determine sample size and understand odds ratio, the historical data was useful. However, in terms of predicting response rate and calculating CLV, the round 1 data will be more reliable.

3. Is there a “best product” that will likely be preferred by all customers? If so, what is it? (3 points) 

Yes, we built a logistic regression model based on round 1 data and found that the product that have the highest response rate among all types of customers is 14.9 APR + Fixed rate + 0 Annual Fee (AKA Offer #3)

4. Describe and justify your testing strategy (10 points) 

We trained a logistic regression model to predict the response rate for each types of customers and for different offers. Then, we used the predicted result of each customer category, compared with the mean of all predicted result, to calculate the required sample size in round 1. After getting the data from round 1, we trained a another logistic model based on that data. And finally, we predicted the response rate again by using this new model and calculated expected CLV by multiplying the response probability with CLV, and select the top two offer, under all three customers groups, that yield the highest expected CLV.

5. Also, try different random seeds (e.g., 19, 1234, or 172110) and discuss the impact of these different settings on the factorial design (are they the same?) and on your subsequent decisions. 

The more interactions we added, the more trails we needed in order to reduce the multicollinearity. Setting different seeds allow us to gain more observations across different sub-sampling choices. When we determine how many trials we need to conduct, different seeds will return the same optimal number of trials. But after we have determined the number of trials, different seeds will return different combinations of trials.

# Report

## Part 1: 

(Logistic Training) 

We trained a logistic regression using exhibit 1 data, which contains variables, apr, fixed_var, annual_fee, bk_score. The model is based on melted exhibit1 dataset which contains original 28 combinations (some not experimented) with their corresponding response. The total training data has 1520000 records, for which each combination frequency is borrowed from the number of responses.  

 

(Prediction & Expected CLV) 

Now, we have a trained logistic model, and we applied this to a full complete combination of apr, fixed_var, annual fee, and bk_score to get a prediction of response probability. Moreover, we joined the exhibit 2 dataset with CLV into the full combination dataset to calculate the expected CLV response. 

 

(Partial Factorial Design) 

A partial factorial design is now introduced in our process. All levels of variables (apr, fixed_var, annual_fee, bk_score) were input into Radiant experiment design with assumablly interactions between annual_fee:apr, apr:fixed_var, and apr:bk_score (We assumed apr has interactions with all other variables). The more interactions we added, the more trails we needed in order to reduce the multicollinearity. Setting different seeds allow us to gain more observations across different sub-sampling choices. When we determine how many trials we need to conduct, different seeds will return the same optimal number of trials. But after we have determined the number of trials, different seeds will return different combinations of trials.
The lowest proper (low collinearity) partial factorial design is 18 trials (combinations) that are sufficient to represent the whole data variation/differences. Thus, we went with these 18 trials for our first round. 

## Part 2: Sample Selection 

In order to simulate the customer response rate on the PFG Bank platform, we need to determine the appropriate sample size. Our team used the “sample size” function on radiant to calculate the proper sample amount. Radiant could create the required sample size by imputing two proportion variables. The first proportion variable is the average of response rate from the logistic regression model. Another proportion is one of the predicted response rates from the 18-factorial design. By repeating the above process 18 time, we get 18 different sample size. Then, we implemented these 18 sample sizes to the simulation platform. However, there are limited trails in the two rounds. Each Bk_score has only 240,000 mailing opportunities. Experimental is expensive and time-consuming, so we should optimize the simulation model by scaling down the sample size if it’s too large.  

From the first-round simulation, we only receive 1.76% response rates from the target customers and the profit is 12,821. Hence, we load the round 1 results back to the logistic regression model to improve the accuracy of the prediction model. 

## Part 3: Build Model Based on Round 1 Data
Since we have the data from round 1 right now, our team can use this data to build another logistic regression model. We first downloaded the result of round 1 from the website, transferred the data format to the same format with exhibit 1, and finally, melted the data so that we can build logistic regression just like we did before. Then, we built a new dataset including all products and predicted the response probability for each BK score. Finally, we calculated expected CLV by multiplying the response probabilities and CLV, and ranked the whole dataset by BK score and expected CLV in descending order, to see what are most optimal products, which can lead to the highest CLV, for different customer groups (different BK score).

## Part 4: Final Selection
After we got the result from the new logistic regression model, we ranked the result based on the predicted response rate and ClV value. Then, we used the predicted response rate times CLV value to get the expected CLV value. For pretending the predicted response rate have minor error and reduce the risk for the competition, we decided to use the top 2 combinations for each BK score. For BK score equals 250, we selected 19.8 apr+Variable+0 annual fees and 14.9 apr+Variable+20 annual fee. For BK score equals 200, we selected 14.9 apr + Variable + 0 annual fee and 14.9 apr + Fixed + 0 annual fee. For BK score equals to 150, we selected 14.9 apr + Variable + 0 annual fee and 14.9 apr + Fixed + 0 annual fee. Then, we calculated the weight by using the expected CLV value. The final step of our calculation is using the weight we got from last step times the remaining mailings for each BK score. The final response rate after round two is 0.034 which is way much higher than the average response rate. And, the profit is 744,445 which is in line with our expectations. 